In [131]:
import pandas
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [182]:
df = pandas.read_csv('./analysis_df.csv')
df_student = df[df['Title'] == 'Student']
df_employee = pandas.read_csv('./analysis_df_employee.csv')

-----
### plot 1

In [183]:
# Get every platfroms from everyone choice
learning_platfroms_employee = df_employee[['Learning Platforms']].apply(lambda x: 
                                                                        list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                                                .explode().value_counts().to_frame(name='Vote').drop(index = 'nan')
learning_platfroms = df_student[['Learning Platforms']].apply(lambda x: 
                                                      list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                                        .explode().value_counts().to_frame(name='Vote').drop(index = 'nan')

In [163]:
# --- drop un useful records
for i in learning_platfroms.index:
    if i.isnumeric():
        learning_platfroms.drop(index = i,inplace=True)
learning_platfroms.drop(index = '-1',inplace=True)

for i in learning_platfroms_employee.index:
    if i.isnumeric():
        learning_platfroms_employee.drop(index = i,inplace=True)
        
learning_platfroms_employee.drop(index = '-1',inplace=True)

# -----

learning_platfroms.reset_index(inplace=True)
learning_platfroms.rename(columns={'index':'Platfrom'},inplace=True)

learning_platfroms_employee.reset_index(inplace=True)
learning_platfroms_employee.rename(columns={'index':'Platfrom'},inplace=True)


In [164]:
# get top 15
learning_platfroms=learning_platfroms.sort_values('Vote',ascending=False).head(15)
learning_platfroms_employee=learning_platfroms_employee.sort_values('Vote',ascending=False).head(15)

In [165]:
fig = make_subplots(rows=1,cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    subplot_titles=("Student", "Employee"))
fig.add_trace(go.Pie(
     name='',
     values=learning_platfroms['Vote'],
     labels=learning_platfroms['Platfrom'],hole=0.3),  
     row=1, col=1)

fig.add_trace(go.Pie(
     name='',
     values=learning_platfroms_employee['Vote'],
     labels=learning_platfroms_employee['Platfrom'],hole=0.3),
    row=1, col=2)

fig.update_layout(height=600, width=1600, title_text="Learning Platforms")

Nhận xét: Với 15 nền tảng khảo sát được chọn nhiều nhất bởi 2 phía đều được giữa nguyên mà không có sự xuất hiện của một nền tảng nào khác đối với bên còn lại, dẫn đầu bởi `Coursera` và `Udemy` đã chiếm một phần phổ biến rất lớn đối với nhưng người được khảo sát khi có đến khoảng 1/3 số người lựa chọn nền tảng này, cho thấy rằng không có sự khác biệt lớn giữa những người đi học về đã đi làm ở khoảng mục này

--- 
### plot 2

In [173]:
# Get every Language from everyone choice
Languages_employee = df_employee[['Languages']].apply(lambda x: 
                                                            list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                                    .explode().value_counts().to_frame(name='Employee')
Languages = df_student[['Languages']].apply(lambda x: 
                                            list( map(lambda y:y.strip(),str(x[0]).split('--'))),axis=1)\
                                            .explode().value_counts().to_frame(name='Student')

In [174]:
# --- drop un useful records
for i in Languages_employee.index:
    if i.isnumeric():
        Languages_employee.drop(index = i,inplace=True)
Languages_employee.drop(index = ['-1','nan'],inplace=True)


for i in Languages.index:
    if i.isnumeric():
        Languages.drop(index = i,inplace=True)
        
Languages.drop(index = ['-1','nan'],inplace=True)

# -----

Languages.reset_index(inplace=True)
Languages.rename(columns={'index':'Languages'},inplace=True)

Languages_employee.reset_index(inplace=True)
Languages_employee.rename(columns={'index':'Languages'},inplace=True)
Languages_use= pandas.merge(Languages,Languages_employee,how='inner')

Languages_use_percent  = Languages_use
Languages_use_percent[['Student','Employee']] =  Languages_use[['Student','Employee']]/Languages_use[['Student','Employee']].sum()

In [175]:
plot = go.Figure(data=[go.Bar(
    name = 'Student',
    x = Languages_use_percent['Languages'],
    y = Languages_use_percent['Student'],
   ),
                       go.Bar(
    name = 'Employee',
    x = Languages_use_percent['Languages'],
    y = Languages_use_percent['Employee']
   )
])  
plot.update_layout(
    xaxis_title="Languages", yaxis_title="Percentage"
)               
plot.show()

Nhận xét: Không lạ khi Python vẫn là lựa chọn hàng đầu của lĩnh vực này, và cũng không có nhiều khác biệt lớn giữa người đi làm và người chưa đi làm, ngoại trừ việc người đi làm ta thấy nhỉnh hơn về số lượng các ngôn ngữ mang chuyên tính chuyên môn 'khá hơn' như R, Bash, SQL, Scala, VBA, ... trong khi những người chưa đi làm thì có xu thế học những ngôn ngữ mang tính 'đề cử' cho người bắt đầu học như java, C hay C++

----
### Plot 3

In [187]:
employee_paper = df_employee['Published Papers'].value_counts().to_frame()
not_employee_paper = df['Published Papers'].value_counts().to_frame() - employee_paper

In [199]:
fig = make_subplots(rows=1,cols=2,
                    specs=[[{"type": "pie"}, {"type": "pie"}]],
                    subplot_titles=("Employee","Unemployment"))
fig.add_trace(go.Pie(
     name='',
     values=employee_paper['Published Papers'],
     labels=employee_paper.index.tolist(),hole=0.3),  
     row=1, col=1)

fig.add_trace(go.Pie(
     name='',
     values=not_employee_paper['Published Papers'],
     labels=not_employee_paper.index.tolist(),hole=0.3),
    row=1, col=2)

fig.update_layout(height=600, width=1000, title_text="Published Papers")

nhìn chung ta có thể thấy được việc có bào báo publish hay không cũng có một phần ảnh hưởng đến khả năng có việc của những người khảo sát khi đa phần những người chưa có việc cũng chưa có bài báo publish, tuy nhiên việc này thì không mang tính bắt buộc vì khi ta xem những khảo sát thì số người có việc thì số lượng người không có bài báo publish cũng chiếm hơn 50%, tuy rằng không chiếm phần lớn như những người chưa có việc